# NHL 2017-2018 Season Player interactive graph using Bokeh. Source http://www.hockeyabstract.com/testimonials/nhl2017-18. Thank you very much to the work of all those involved at Hockey Abstract for their work and sharing of this feature rich dataset.

In [1]:
import pandas as pd
import numpy as np
from math import pi
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('nhl_2017_18_condensed.csv')

In [9]:
#Import the required libraries
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, Slider, CategoricalColorMapper, Select,LabelSet, TextInput, Range1d
from bokeh.layouts import widgetbox,row, column

Loading BokehJS ...

### The below cell walks through the steps we took to create an interactive graph using a cut down feature set from Hockey Abstract data. The data is stored in the file 'nhl_2017_18_condensed.csv'. We personally flattened the dataset from Hockey Abstract and manipulated the data into its current form. If you use this CSV file, or plot, please credit this source as well as those at Hockey Abstract. Run the cell below and explore the different relationships and visualizations it can create.

### Any constructive feedback or suggestions are welcome. Contact info is williamzdupree.com or willdupree90@gmail.

In [8]:
#This function will push the bokeh plot with updateds
def modify_doc(doc):
    
    #Set default players to include in top look-up
    player_count = 6
    
    # Make the ColumnDataSource for main graph: source
    source = ColumnDataSource(data={
        'x':df.sort_values(by=['PlInf_Wt'],ascending = False).iloc[:player_count]['PlInf_Name'],
        'y':df.sort_values(by=['PlInf_Wt'],ascending = False).iloc[:player_count]['PlInf_Wt'],
            })
    
    ####################################
    #Make the widgets that will go into our interactive graph
       
    # Make a slider object: slider
    slider = Slider(start=1,end = 10,step =1,value=player_count,title='Number of Top Players')
    
    # Create a dropdown Select widget for the y data: y_select
    
    # Dictionary to help map between abbreviations in column titles and user friendly strings
    abrev_bokeh_dict_1 = {'ContInf_Salary':['Salary','Dollars ($)'],'PlInf_Ht':['Height','Inches'],'PlInf_Wt':['Weight','Lbs.'],
                   'PlInf_Age':['Age','Years'],'PlInf_Seasons':['Seasons Played','Number'],'PrimStat_GP':['Games Played','Number'],
                    'PrimStat_G':['Goals Scored','Number'],'PrimStat_A':['Assists Made','Number'],
                    'PrimStat_+/-':['Plus-Minus','Number'],'PrimStat_PIM':['Penalties in Minutes','Minutes'],
                    'PrimStat_Shifts':['Shifts Played','Number'],'PrimStat_TOI':['Time On Ice','Seconds'],
                    'IndStat_iSF':['Shots on Goal','Number'],'PenStat_iPenT':['Penalties Taken','Number'],
                    'PenStat_iPenD':['Penalties Drawn','Number'],'CarStat_GP':['Career Total Games Played','Number'],
                   'CarStat_G':['Career Total Goals','Number'],'CarStat_A':['Career Total Assists','Number'],
                    'CarStat_+/-':['Career Total Plus-Minus','Number'],'CarStat_PIM':['Career Total Penalties in Minutes','Number'],
                    'CarStat_TOI':['Career Total Time on Ice','Minutes']}
    
    #It will be slightly confusing, but for user's ease we will flip flop our dictionary so that the drop down
    #menu makes better sense of our options to choose from
    non_abrev = [item[0] for item in abrev_bokeh_dict_1.values() ]
    is_abrev = [item for item in abrev_bokeh_dict_1.keys()]
    
    #Creat the second dictionary where we have reversed our key value pair. The reasoning will be so that a 
    #user can make sense of our drop down menu, then Bokeh will update the value from mapping to the dictionary. This
    #Value is a column of our dataframe. Finally, when updating we use the reverse mapping to call values from the dataframe
    #column name to get the plot label and axes label
    abrev_bokeh_dict_2 = dict(zip(non_abrev,is_abrev))
    
    y_select = Select(
        options=non_abrev,
        value='Weight',
        title='Desired Stat'
    )
    
    
    #######################################
    #We now must define callback functions for our varius widgets
    
    # Define the callback function: update_plot
    def update_plot(attr, old, new):
        # Read the current value off the slider and dropdown: num and y
        num = slider.value
        y = abrev_bokeh_dict_2[y_select.value]
        
        new_data = {'x':df.sort_values(by=[y],ascending = False).iloc[:num]['PlInf_Name'],
                    'y':df.sort_values(by=[y],ascending = False).iloc[:num][y],
                   }
        # Assign new_data to source.data
        source.data = new_data

        # Add title to plot
        plot.title.text = 'NHL 2017-2018 top {} player stats for {}'.format(num,abrev_bokeh_dict_1[y][0])
        
        # update axes labels to plot
        plot.yaxis.axis_label = '{}'.format(abrev_bokeh_dict_1[y][1])
        
        plot.x_range.factors = list(source.data['x'])
        
        low = df.sort_values(by=[y],ascending = False).iloc[:num][y].min()
        high = df.sort_values(by=[y],ascending = False).iloc[:num][y].max()
        plot.y_range.start = low - low*.05
        plot.y_range.end = high + high*.05
    
    
    ####################################################
    #We must now call all of our change values incase one of the widget value changes,
    #here we can update both the graph and axes so that whenever any widget changes our
    #plot updates
    
    # Attach the callback to the 'value' property of slider
    slider.on_change('value',update_plot)
    
    # Attach the update_plot callback to the 'value' property of y_select
    y_select.on_change('value',update_plot)
    
    
    # Create the figure: plot
    plot = figure(x_range = source.data['x'] ,plot_height=600,plot_width=600, 
                  title="NHL 2017-2018 top 6 player stats for {}".format(abrev_bokeh_dict_1['PlInf_Wt'][0]),
                  toolbar_location=None, tools="")

    # Add the color mapper to the circle glyph
    p1 = plot.vbar(x='x', top='y', width=0.5,source=source)
    
    # Set the y-axis label
    plot.yaxis.axis_label = '{}'.format(abrev_bokeh_dict_1['PlInf_Wt'][1])
    
    #Set y-axis range
    min_y = df.sort_values(by=['PlInf_Wt'],ascending = False).iloc[:6]['PlInf_Wt'].min()
    max_y = df.sort_values(by=['PlInf_Wt'],ascending = False).iloc[:6]['PlInf_Wt'].max()
    plot.y_range = Range1d(min_y - min_y*.05,max_y + max_y*.05)
    plot.xaxis.major_label_orientation = pi/6
    
    # Set Label sizes
    plot.title.text_font_size = '15pt'
    
    plot.yaxis.major_label_text_font_size = '12pt'
    plot.xaxis.major_label_text_font_size = '12pt'

    plot.xaxis.axis_label_text_font_size = '12pt'
    plot.yaxis.axis_label_text_font_size = '12pt'
    
    ####################
    #Set the layout for the plot so it is visually pleasing.
    layout = column(row(widgetbox(slider, y_select),plot))
    
    #Set the doc and then show the modified document, having an interactive graph
    doc.add_root(layout)
show(modify_doc)